In [9]:
from scipy.special import jv
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi
import pandas as pd
import matplotlib as mpl
from floquet_simulations.flux_functions import *
from pathlib import Path
from matplotlib.colors import LinearSegmentedColormap
from floquet_simulations.plot_functions import PlotParams
from scipy.special import jn_zeros
from floquet_simulations.hamiltonians import ConvertComplex

In [10]:
omega0 = 8
alpha=1; beta=2
data_dir = Path().absolute().parent.parent/"paper_data"/f"Heff_omega={omega0},alpha={alpha},beta={beta}.csv"
df = pd.read_csv(data_dir,
                  index_col=False,
                   converters={"FT-J12": ConvertComplex,
                                  "FT-J23": ConvertComplex,
                                  "FT-J31": ConvertComplex,
                   }
                  )


In [11]:

df["phi3/pi"]= np.round(df["phi3/pi"], 2)
df["xi"] = np.angle(df["FT-J23"].to_numpy()*df["FT-J12"].to_numpy()*df["FT-J31"].to_numpy())

n_xi0s = 51
xis = np.linspace(0,1, n_xi0s)
r10 = 1
r20 = 1
ix = -1
delta = 0.08
df_targets = pd.DataFrame(columns=["r10", "r20", "xi0_frac", "delta", "A2", "A3", "omega", "varphi_frac"])

i_start = (200,0)
for ii, xi0 in enumerate(xis):
    dfn = df[(df.xi <xi0*pi+delta)
            &(df.xi > xi0*pi - delta)
            &(df["FT-LowerT.X"] > r10-delta)
            &(df["FT-LowerT.X"] < r10+delta)
            &(df["FT-LowerT.Y"]> r20-delta)
            & (df["FT-LowerT.Y"] < r20+delta)
            ]
    print(len(dfn))
    if len(dfn) < i_start[0]:
        i_start = (len(dfn), ii)
    new_row = pd.DataFrame({'r10':[r10], 'r20':[r20], 'xi0_frac':[xi0], 'delta':[delta], "A2":[dfn.A2.iloc[ix]], "A3":[dfn.A3.iloc[ix]], "omega":[dfn.omega0.iloc[ix]], "varphi_frac":[dfn["phi3/pi"].iloc[ix]]})
#     print(dfn.A3.to_list())
    df_targets = pd.concat([df_targets, new_row], ignore_index=True)
print(f"Min dfn size is {i_start[0]} at index {i_start[1]}")

20810
20398
604
322
208
128
100
72
48
44
86
120
134
146
182
198
180
180
203
246
285
258
226
220
186
124
80
96
86
62
18
10
12
10
4
2
4
4
2
2
2
2
2
8
22
48
78
107
90
53
26
Min dfn size is 2 at index 35


In [12]:
# i_start = (2,45)
# dfn = df[(df.xi < xis[i_start[1]]*pi+delta)
#             &(df.xi > xis[i_start[1]]*pi - delta)
#             &(df["FT-LowerT.X"] > r10-delta)
#             &(df["FT-LowerT.X"] < r10+delta)
#             &(df["FT-LowerT.Y"] > r20-delta)
#             & (df["FT-LowerT.Y"] < r20+delta)
#             ]

In [13]:
# we have the index of the starting value. make a smart df and start from the hardest point
df_targets_smart = pd.DataFrame(columns=["r10", "r20", "xi0_frac", "delta", "A2", "A3", "omega", "varphi_frac"])

st_ind = i_start[1]
df_st = df[(df.xi < xis[st_ind]*pi+delta)
            &(df.xi > xis[st_ind]*pi - delta)
            &(df["FT-LowerT.X"] > r10-delta)
            &(df["FT-LowerT.X"] < r10+delta)
            &(df["FT-LowerT.Y"]> r20-delta)
            & (df["FT-LowerT.Y"] < r20+delta)
            ]
A2_ind = np.real(df_st.A2.iloc[0])
A3_ind = np.real(df_st.A3.iloc[0])
varphi_ind = np.real(df_st["phi3/pi"].iloc[0])

new_row = pd.DataFrame({'r10':[r10], 'r20':[r20], 'xi0_frac':[ xis[st_ind]], 'delta':[delta], 
                            "A2":[A2_ind],
                             "A3":[A3_ind],
                               "omega":[8],
                              "varphi_frac":[varphi_ind]})
#     print(dfn.A3.to_list())
df_targets_smart = pd.concat([df_targets_smart, new_row], ignore_index=True)


In [14]:
for ind in reversed(range(st_ind)):
    df_ind = df[(df.xi < xis[ind]*pi+delta)
            &(df.xi > xis[ind]*pi - delta)
            &(df["FT-LowerT.X"] > r10-delta)
            &(df["FT-LowerT.X"] < r10+delta)
            &(df["FT-LowerT.Y"]> r20-delta)
            & (df["FT-LowerT.Y"] < r20+delta)
            ]
    #set placeholder for the diff
    diff = (10000, -1)
    for row_i in range(len(df_ind)):
        new_diff =  (np.real(df_ind.iloc[row_i].A2) - A2_ind)**2 + (np.real(df_ind.iloc[row_i].A3) - A3_ind)**2 + 0.1*(np.real(df_ind.iloc[row_i]["phi3/pi"]) - varphi_ind)**2 
        new_diff = new_diff + ((np.real(df_ind.iloc[row_i].xi) - xi0*pi)**2 + (np.real(df_ind["FT-LowerT.X"].iloc[row_i]) - r10)**2 + (np.real(df_ind["FT-LowerT.Y"].iloc[row_i]) -  r20)**2)
        if new_diff < diff[0]:
            diff = (new_diff, row_i)
            
    A2_ind = np.real(df_ind.A2.iloc[diff[1]])
    A3_ind = np.real(df_ind.A3.iloc[diff[1]])
    varphi_ind = np.real(df_ind["phi3/pi"].iloc[diff[1]])

    new_row = pd.DataFrame({'r10':[r10], 'r20':[r20], 'xi0_frac':[ xis[ind]], 'delta':[delta], 
                            "A2":[A2_ind],
                                "A3":[A3_ind],
                                "omega":[8],
                                "varphi_frac":[varphi_ind]})
    #     print(dfn.A3.to_list())
    df_targets_smart = pd.concat([df_targets_smart, new_row], ignore_index=True)

In [15]:
# get start point of initial start point
A2_ind = np.real(df_st.A2.iloc[0])
A3_ind = np.real(df_st.A3.iloc[0])
varphi_ind = np.real(df_st["phi3/pi"].iloc[0])


for ind in range(st_ind+1, n_xi0s):
    df_ind = df[(df.xi < xis[ind]*pi+delta)
            &(df.xi > xis[ind]*pi - delta)
            &(df["FT-LowerT.X"] > r10-delta)
            &(df["FT-LowerT.X"] < r10+delta)
            &(df["FT-LowerT.Y"]> r20-delta)
            & (df["FT-LowerT.Y"] < r20+delta)
            ]
    #set placeholder for the diff
    diff = (1000, -1)
    for row_i in range(len(df_ind)):
        new_diff =  (np.real(df_ind.iloc[row_i].A2) - A2_ind)**2 + (np.real(df_ind.iloc[row_i].A3) - A3_ind)**2 + 0.1*(np.real(df_ind.iloc[row_i]["phi3/pi"]) - varphi_ind)**2 
        new_diff = new_diff + 100*((np.real(df_ind.iloc[row_i].xi) - xi0*pi)**2 + (np.real(df_ind["FT-LowerT.X"].iloc[row_i]) - r10)**2 + (np.real(df_ind["FT-LowerT.Y"].iloc[row_i]) -  r20)**2)

        if new_diff < diff[0]:
            diff = (new_diff, row_i)
            
    A2_ind = np.real(df_ind.A2.iloc[diff[1]])
    A3_ind = np.real(df_ind.A3.iloc[diff[1]])
    varphi_ind = np.real(df_ind["phi3/pi"].iloc[diff[1]])

    new_row = pd.DataFrame({'r10':[r10], 'r20':[r20], 'xi0_frac':[ xis[ind]], 'delta':[delta], 
                            "A2":[A2_ind],
                                "A3":[A3_ind],
                                "omega":[8],
                                "varphi_frac":[varphi_ind]})
    #     print(dfn.A3.to_list())
    df_targets_smart = pd.concat([df_targets_smart, new_row], ignore_index=True)

In [16]:
df_targets_smart = df_targets_smart.sort_values(by=['xi0_frac'])
df_targets_smart.to_csv(Path().absolute().parent.parent/"paper_data"/f"numerical_optimisation"/"initial_conditions_r1,r2=1,1.csv", index=False)